Connecting to WRDS

In [ ]:

import os
import wrds
import pandas as pd
import sqlite3

wrds_username = os.environ.get("WRDS_USERNAME")
wrds_password = os.environ.get("WRDS_PASSWORD")

db = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)


# Sample query

In [ ]:

# Retrieve 2015 - 2017 Earnings Conference Call Transcripts with Full-text components
sql_query = '''SELECT a.*, b.*, c.componenttext \
                 FROM (SELECT * FROM ciq.wrds_transcript_detail \
                                WHERE companyid in (112350, 21835, 24937) AND \
                                      date_part('year',mostimportantdateutc)>=2015 AND \
                                      date_part('year',mostimportantdateutc)<=2017) AS a, \
                      ciq.wrds_transcript_person as b, ciq.ciqtranscriptcomponent AS c \
                 WHERE a.transcriptid=b.transcriptid AND b.transcriptcomponentid=c.transcriptcomponentid \
                 ORDER by a.transcriptid, b.componentorder;'''
 
print(sql_query) 
data = db.raw_sql(sql_query)

In [ ]:
data.columns

# Full query: year by year from 2000 to 2023

In [ ]:

# Use SQLLite to store the data
# Create a connection to the SQLite database (it will create a new one if it doesn't exist)
conn = sqlite3.connect('../data/transcripts_raw_v2/transcripts_data.db')


In [ ]:


for y in range(2023, 2024):
    print( f'Processing year {y}')
    sql_query = f'''SELECT a.*, b.*, c.componenttext \
                    FROM (SELECT * FROM ciq.wrds_transcript_detail \
                                    WHERE date_part('year',mostimportantdateutc)= {y}) AS a \
                        JOIN ciq.wrds_transcript_person as b ON a.transcriptid=b.transcriptid \
                        JOIN ciq.ciqtranscriptcomponent AS c ON b.transcriptcomponentid=c.transcriptcomponentid \
                    ORDER by a.transcriptid, b.componentorder;'''

    # Fetch data directly into the SQLite database
    data = db.raw_sql(sql_query)
    data.to_sql(f'transcripts_{y}', conn, if_exists='replace', index=False)  # Name the table based on the year

# Commit changes and close connection
conn.commit()
conn.close()


## Testing the tables to make sure it works

In [ ]:
# Create a connection to the SQLite database (it will create a new one if it doesn't exist)
# list the tables in the database
conn = sqlite3.connect('../data/transcripts_raw_v2/transcripts_data.db')

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
# print the number of rows in each table   
for table in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall():
    print(table[0], cursor.execute(f"SELECT count(*) FROM {table[0]};").fetchall()[0][0])

# Downloading identifier tables

In [ ]:
# list of tables to download:
id_tables = ['wrds_cik', 'wrds_cusip', 'wrds_gvkey', 'wrds_ciqsymbol']

In [ ]:
# create a new database named 'id_data.db'
# download each table into the database as a separate table
# use the same table name as the original table name

conn = sqlite3.connect('../data/transcripts_raw_v2/id_data.db')

for table in id_tables:
    print(f'Downloading table {table}')
    sql_query = f'''SELECT * FROM ciq.{table};'''
    # Fetch data directly into the SQLite database
    data = db.raw_sql(sql_query)
    data.to_sql(f'{table}', conn, if_exists='replace', index=False)  # Name the table based on the year


# Commit changes and close connection
conn.commit()
conn.close()
